In [9]:
import os 
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [27]:
from langchain_community.tools import DuckDuckGoSearchRun

tool1 = DuckDuckGoSearchRun()
tool1.invoke("Tell me about todays new in India?")

"India News | Latest India News | Read latest and breaking news from India. Today's top India news headlines, news on Indian politics, elections, government, business, technology, and Bollywood. Today's Latest News: Read Latest Big news headlines from India and around the world with live coverage on India Today. ... Rishabh Pant opened up on his relationship with new India Test captain Shubman Gill and said that he gels really well with the 25-year-old on and off the field. Pant will act as Gill's deputy as he begins his time as the ... Israel-Iran Conflict Today LIVE News Updates: Trump posted that Israel now has 'complete and total control of the skies over Iran' Updated: June 18, 2025 01:30 IST 'Mohalla clinics promoted corruption, not public health': Delhi CM at launch of 33 Arogya Mandirs; AAP hits back Today's Latest News: Get the news headlines from India and around the world on Oneindia.com. Stay updated with today's news, live coverage, videos, and photos across politics, ente

In [10]:
#This tool is used to run the command on our command line 
from langchain_community.tools import ShellTool

tool2 = ShellTool()
print(tool2.invoke("dir"))

Executing command:
 dir
 Volume in drive C is Acer
 Volume Serial Number is E464-3CD8

 Directory of c:\Users\Adwait\Desktop\AIML\Exploring-Langchain\Agents

20-06-2025  12:37    <DIR>          .
05-05-2025  20:28    <DIR>          ..
20-09-2024  10:45         2,215,244 attention.pdf
19-06-2025  16:39            10,250 currency_exchange_tool.ipynb
03-06-2025  10:41             1,038 current_wether_update.ipynb
20-06-2025  10:08            16,887 learning_tools_in_detail.ipynb
20-06-2025  12:30             1,518 test.py
19-06-2025  17:11            12,672 tools.ipynb
19-06-2025  17:18             6,732 tool_calling.ipynb
               7 File(s)      2,264,341 bytes
               2 Dir(s)  17,566,564,352 bytes free



c:\Users\Adwait\Desktop\AIML\Exploring-Langchain\env\lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.7
)

In [19]:
from langchain_community.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Given two numbers a and b this tool returns their product"""
    return a * b

@tool
def text(a: int) -> str:
    "You will be given a number and your task will be to return the english spelling of that number"
    return f"return the english spelling of {a} number"

In [20]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [21]:
loader = PyPDFLoader('attention.pdf')
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents = text_splitter.split_documents(docs)
db = FAISS.from_documents(documents, GoogleGenerativeAIEmbeddings(model="models/embedding-001"))
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000252A47C18A0>, search_kwargs={})

In [22]:
from langchain.tools import Tool
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

Tool(name='PDF_QA', description='Useful when you want to answer questions from the PDF document.', func=<bound method Chain.run of RetrievalQA(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]), llm=ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generat

In [ ]:
llm_tool = llm.bind_tools([multiply, text])

In [25]:
response = llm_tool.invoke("Hey how are you my name is adwait my I know your name?")
response.content

"I am a large language model. I don't have a name."

In [26]:
response = llm_tool.invoke("Can you multiply 2 with 8")
print(response)

content='' additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 8.0}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--b98bcedd-501a-43e3-a0e7-1661f102f995-0' tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 8.0}, 'id': '13028126-e117-4224-9c07-a701f16ed579', 'type': 'tool_call'}] usage_metadata={'input_tokens': 74, 'output_tokens': 5, 'total_tokens': 79, 'input_token_details': {'cache_read': 0}}


In [27]:
response = llm_tool.invoke("Can you tell me the english spelling of 2")
print(response)

content='' additional_kwargs={'function_call': {'name': 'text', 'arguments': '{"a": 2.0}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--8c32c377-9b8b-41ab-9e0b-6506c1df3b48-0' tool_calls=[{'name': 'text', 'args': {'a': 2.0}, 'id': '391c32a9-09a1-48fe-9e6e-1503d2392cb9', 'type': 'tool_call'}] usage_metadata={'input_tokens': 76, 'output_tokens': 3, 'total_tokens': 79, 'input_token_details': {'cache_read': 0}}


In [28]:
response = llm_tool.invoke("Can you multiple 2 with 6 and then tell me the english spelling of that")
print(response)

content='' additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 6.0}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--7d6f0e27-16da-4596-b1fb-a68cdd9efd8a-0' tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 6.0}, 'id': 'fe199bcc-7ed1-4609-a6ec-05abafbb638d', 'type': 'tool_call'}] usage_metadata={'input_tokens': 83, 'output_tokens': 5, 'total_tokens': 88, 'input_token_details': {'cache_read': 0}}


In [29]:
response.tool_calls

[{'name': 'multiply',
  'args': {'a': 2.0, 'b': 6.0},
  'id': 'fe199bcc-7ed1-4609-a6ec-05abafbb638d',
  'type': 'tool_call'}]

So the conclussion for the above code is that when we are binding tool with a LLM then at that time it LLM is only responsible to to call one tool it does not has the capability to chain the tools ony by one to provide the output

In [50]:
from langchain.memory import ConversationBufferMemory

# --- Add Memory ---
memory = ConversationBufferMemory(
    memory_key="chat_history",  
    return_messages=True         
)

In [32]:
tools = [multiply,
                text,
                Tool(
                    name="PDF_QA",
                    func=qa_chain.run,
                    description="Useful when you want to answer questions from the PDF document."
)]

In [33]:
from langchain.agents import AgentType, initialize_agent

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    # memory=memory,
    verbose=True
)

In [37]:
response = agent.invoke("""Tell me what is self attention""")
print(response)



> Entering new AgentExecutor chain...
Thought: I need to use the PDF_QA tool to answer the question about self-attention.
Action:
```
{
  "action": "PDF_QA",
  "action_input": "What is self-attention?"
}
```
Observation: Self-attention, sometimes called intra-attention, is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence.
Thought:I have the answer to the question.
Action:
```
{
  "action": "Final Answer",
  "action_input": "Self-attention, sometimes called intra-attention, is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence."
}
```

> Finished chain.
{'input': 'Tell me what is self attention', 'output': 'Self-attention, sometimes called intra-attention, is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence.'}
